In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#config = tf.compat.v1.ConfigProto
#config.gpu_options.allow_growth = True
#session = tf.compat.v1.Session(config=config)
#tf.compat.v1.keras.backend.set_session(session)

In [9]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

#model.summary()

NotImplementedError: Cannot convert a symbolic Tensor (lstm_2/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [7]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 28

units = 64
output_size = 10  # labels are from 0 to 9

# Build the RNN model   
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(None, input_dim)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [8]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
sample, sample_label = x_train[0], y_train[0]

In [9]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)


model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=5
)

Epoch 1/5
938/938 [==============================] - 15s 16ms/step - loss: 0.9672 - accuracy: 0.6898 - val_loss: 0.5624 - val_accuracy: 0.8162
Epoch 2/5
938/938 [==============================] - 12s 13ms/step - loss: 0.4124 - accuracy: 0.8740 - val_loss: 0.3656 - val_accuracy: 0.8816
Epoch 3/5
938/938 [==============================] - 14s 15ms/step - loss: 0.2634 - accuracy: 0.9225 - val_loss: 0.2828 - val_accuracy: 0.9046
Epoch 4/5
938/938 [==============================] - 14s 14ms/step - loss: 0.2020 - accuracy: 0.9403 - val_loss: 1.1896 - val_accuracy: 0.6450
Epoch 5/5
938/938 [==============================] - 14s 15ms/step - loss: 0.1693 - accuracy: 0.9490 - val_loss: 0.2030 - val_accuracy: 0.9314


In [7]:
model.save('F:\PycharmProjects\lab_9\mnist_lab_9.h5')
print("Модель сохранена как mnist_lab_9.h5")

Модель сохранена как mnist.h5
